# Local Agents: From Structured Outputs to Tool Calling

In this notebook, we'll build up to creating a local AI agent step by step:

1. **Structured Outputs** - Getting reliable JSON from LLMs using Pydantic
2. **Tool/Function Calling** - Having the LLM decide which functions to call
3. **Complete Agent Loop** - Putting it all together into an agentic workflow

All examples run locally using [Ollama](https://ollama.ai/).

In [ ]:
!pip install -U ollama pydantic

---
## Part 1: Structured Outputs with Pydantic

The foundation of reliable agents is getting **structured, predictable outputs** from LLMs.

Instead of parsing free-form text, we can force the LLM to return valid JSON that matches our schema.

In [ ]:
from ollama import chat
from pydantic import BaseModel
from typing import Optional, List

### Example 1: Task Generation

Let's define a `Task` model and have the LLM generate a realistic task:

In [ ]:
class Task(BaseModel):
    title: str
    description: str
    priority: str  # "High", "Medium", "Low"
    estimated_hours: float
    dependencies: List[str] = []
    status: str = "Not Started"  # "Not Started", "In Progress", "Completed"
    assigned_to: Optional[str] = None

In [ ]:
response = chat(
    messages=[
        {
            'role': 'user',
            'content': '''Create a task for implementing a new feature in our project management software 
            that allows users to track time spent on tasks. Include dependencies and make it realistic.''',
        }
    ],
    model='mistral-small3.2',
    format=Task.model_json_schema(),  # This forces structured output!
)

# Validate and parse the response
task = Task.model_validate_json(response['message']['content'])
task

In [ ]:
print(f"Task: {task.title}")
print(f"Priority: {task.priority}")
print(f"Estimated Hours: {task.estimated_hours}")
print(f"Dependencies: {', '.join(task.dependencies)}")
print(f"Status: {task.status}")

### Example 2: Email Tool Input

Structured outputs are perfect for extracting tool inputs from natural language:

In [ ]:
class EmailToolInput(BaseModel):
    email_destination: str
    email_contents: str


prompt = """
Write an email to my boss: boss_of_lucas@gmail.com, telling him that I quit to pursue bird watching.
"""

email_response = chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': prompt}],
    format=EmailToolInput.model_json_schema(),
)

email_input = EmailToolInput.model_validate_json(email_response['message']['content'])
print(f"To: {email_input.email_destination}")
print(f"\n{email_input.email_contents}")

In [ ]:
# Now we could use this structured data to actually send an email!
def send_email(email_destination: str, email_contents: str):
    print("[SIMULATED] Sending email...")
    print(f"To: {email_destination}")
    print(f"Contents: {email_contents}")

send_email(email_input.email_destination, email_input.email_contents)

---
## Part 2: Native Tool/Function Calling

Ollama supports **native tool calling** - the LLM can decide which function to call and with what arguments.

This is more powerful than structured outputs because the LLM chooses the tool dynamically.

In [ ]:
import ollama

### Define a Tool

Let's create a simple tool that reads information from a file:

In [ ]:
# First, create a sample file with some information
with open("lucas_secrets.txt", "w") as f:
    f.write("""Name: Lucas Soares
Profession: Software Engineer
Favorite Movie: Inception
Favorite Book: The Name of the Wind""")

print("Created lucas_secrets.txt")

In [ ]:
def get_lucas_info(file_path: str) -> str:
    """Read information about Lucas from a file."""
    with open(file_path, "r") as file:
        return file.read()

# Test it
get_lucas_info("lucas_secrets.txt")

### Let the LLM Decide to Use the Tool

In [ ]:
input_prompt = """
What is Lucas's profession, favorite movie, and favorite book?
Use the information from the file: lucas_secrets.txt
"""

response = ollama.chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': input_prompt}],
    tools=[{
        'type': 'function',
        'function': {
            'name': 'get_lucas_info',
            'description': 'Get information about Lucas from a file',
            'parameters': {
                'type': 'object',
                'properties': {
                    'file_path': {
                        'type': 'string',
                        'description': 'The path to the file containing information about Lucas',
                    },
                },
                'required': ['file_path'],
            },
        },
    }],
)

print("Tool calls requested by LLM:")
print(response['message']['tool_calls'])

The LLM understood the request and decided to call `get_lucas_info` with the correct file path!

---
## Part 3: Complete Agent Loop

Now let's put it all together into a complete **agentic workflow**:

```
1. User provides input
2. LLM decides which tool(s) to call
3. We execute the tool(s)
4. We feed the results back to the LLM
5. LLM generates the final response
```

In [ ]:
# Tool registry - maps function names to actual functions
TOOLS = {
    'get_lucas_info': get_lucas_info,
}

def execute_tool_call(tool_call):
    """Execute a tool call and return the result."""
    func_name = tool_call['function']['name']
    func_args = tool_call['function']['arguments']
    
    if func_name in TOOLS:
        return TOOLS[func_name](**func_args)
    else:
        return f"Error: Tool '{func_name}' not found"

In [ ]:
def run_agent(user_input: str, tools_schema: list) -> str:
    """
    Run a simple agent loop:
    1. Send user input to LLM with available tools
    2. If LLM requests tool calls, execute them
    3. Send tool results back to LLM for final response
    """
    print(f"\n{'='*50}")
    print(f"User: {user_input}")
    print(f"{'='*50}\n")
    
    # Step 1: Initial LLM call
    response = ollama.chat(
        model='mistral-small3.2',
        messages=[{'role': 'user', 'content': user_input}],
        tools=tools_schema,
    )
    
    # Step 2: Check if tools were called
    tool_calls = response['message'].get('tool_calls', [])
    
    if not tool_calls:
        # No tools needed, return direct response
        return response['message']['content']
    
    # Step 3: Execute tool calls
    print("Agent is calling tools...")
    tool_results = []
    for tool_call in tool_calls:
        func_name = tool_call['function']['name']
        print(f"  -> Calling: {func_name}")
        result = execute_tool_call(tool_call)
        tool_results.append(result)
        print(f"  <- Result: {result[:100]}..." if len(result) > 100 else f"  <- Result: {result}")
    
    # Step 4: Send results back to LLM
    combined_results = "\n".join(tool_results)
    followup_prompt = f"""
The user asked: {user_input}

You called tools and got these results:
{combined_results}

Now provide a helpful response to the user based on this information.
"""
    
    final_response = ollama.chat(
        model='mistral-small3.2',
        messages=[{'role': 'user', 'content': followup_prompt}],
    )
    
    return final_response['message']['content']

In [ ]:
# Define our tools schema
tools_schema = [{
    'type': 'function',
    'function': {
        'name': 'get_lucas_info',
        'description': 'Get information about Lucas from a file',
        'parameters': {
            'type': 'object',
            'properties': {
                'file_path': {
                    'type': 'string',
                    'description': 'The path to the file containing information about Lucas',
                },
            },
            'required': ['file_path'],
        },
    },
}]

# Run the agent!
result = run_agent(
    "What is Lucas's profession, favorite movie, and favorite book? Check lucas_secrets.txt",
    tools_schema
)

print(f"\n{'='*50}")
print("Final Response:")
print(f"{'='*50}")
print(result)

---
## Summary

We've covered the building blocks of local AI agents:

| Concept | Purpose | Ollama Feature |
|---------|---------|----------------|
| **Structured Outputs** | Get predictable JSON from LLMs | `format=Schema.model_json_schema()` |
| **Tool Calling** | Let LLM decide which functions to call | `tools=[...]` parameter |
| **Agent Loop** | Complete workflow with tool execution | Combine both + execution logic |

### Key Takeaways:

1. **Pydantic + `format`** = Reliable structured outputs
2. **`tools` parameter** = LLM-driven function calling
3. **Agent loop** = User input -> LLM -> Tools -> LLM -> Response

This pattern scales to more complex agents with multiple tools, memory, and planning capabilities!